Необходимо выделить в сообщении непустую часть
Разделить ее на фреймы
Подготовить датасет с именами файлов и номерами фреймов

In [46]:
import os
from workWithAudio import *
import pandas as pd

frameLength = 200
modelPath = 'D:\\SD\\yandexML\\pandModel\\models\\model_new_ExtraTreesClassifier_200_9788.sav'
# modelPath = 'D:\\SD\\yandexML\\pandModel\\models\\model_ExtraTreesClassifier_60_9656.sav'
targetsPath = 'D:\\SD\\yandexML\\pandModel\\data\\test'

resp = 'D:\\SD\\yandexML\\pandModel\\data\\testResults2.tsv'
# targetsPath = 'D:\\SD\\yandexML\\pandModel\\data\\extra'
files = os.listdir(targetsPath)
i = 0
dfRes = pd.DataFrame(columns=['file', 'frame'] + [fr"spectr_{jj}" for jj in range(128*frameLength)])

for file in files:
    i+=1
    if(i%100 == 0):
        print(fr"Обработано {i} аудиозаписей")
    
    newAudio = workWithAudio(targetsPath, file)
    [st, fin] = newAudio.getTiming(newAudio.tact)
    croppedAudio = newAudio.tact.T[st: fin]
    l = len(croppedAudio)

    finSpectrList = []

    if(l<frameLength):
        res = np.concatenate([croppedAudio, np.zeros((frameLength-l, 128), float)]).reshape(-1)
        # finSpectrList.append(res)
        dfRes.loc[len(dfRes.index)]= np.append([file, 0], res)
    else:
        for jj in range(0,l-frameLength+1,frameLength):
            res = croppedAudio[jj:jj+frameLength].reshape(-1)
            # finSpectrList.append(res)
            dfRes.loc[len(dfRes.index)] = np.append([file, int(jj/frameLength)], res)
            # print(fr"Срезаем из {l} промежуток от {jj} до {jj+frameLength}")
            # print(res.shape)
        
        if(l%frameLength!=0 and frameLength//(l%frameLength)<4):
            res = croppedAudio[l-frameLength:].reshape(-1)
            # finSpectrList.append(res)
            dfRes.loc[len(dfRes.index)] = np.append([file, int(jj/frameLength)+1], res)
            # print(fr"Срезаем из {l} промежуток от {l-frameLength} до {l}")
            # print(res.shape)
    
dfRes.head()

,file,frame,spectr_0,spectr_1,spectr_2,spectr_3,spectr_4,spectr_5,spectr_6,spectr_7,...,spectr_25590,spectr_25591,spectr_25592,spectr_25593,spectr_25594,spectr_25595,spectr_25596,spectr_25597,spectr_25598,spectr_25599
0,00cb9a6c7a53d779309f209989546013_0.wav,0,7.29131861589849e-05,0.0003721802495419979,0.0009862571023404598,0.0028774456586688757,0.005908974912017584,0.003580308286473155,0.00428567361086607,0.0027748667635023594,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01d0876db1e160a08c829b60362cd067_0.wav,0,0.00013269709597807378,0.00013220659457147121,0.000153780696564354,0.0003066536446567625,0.0004898545448668301,0.00017926160944625735,8.186303602997214e-05,0.00041142432019114494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0a8622ed3cf39e615cafea99a892b901_0.wav,0,0.00023896453785710037,0.00017984920123126358,0.00017556919192429632,6.676952762063593e-05,0.00033841870026662946,0.0021137946750968695,0.0036855002399533987,0.0006243610987439752,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0b07a079f09b95ed62ed5b47d78c271e_0.wav,0,1.739150502544362e-05,0.00012467514898162335,0.00017628076602704823,0.00018073891988024116,0.0002758655173238367,0.00037695933133363724,0.00045939793926663697,0.00012934613914694637,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0b68ceee76b45ba2835a6d571af487e3_0.wav,0,3.4681372198974714e-05,0.00028795559774152935,0.0006027070339769125,0.00033711965079419315,0.00029673337121494114,0.0003502953622955829,0.0002962294383905828,0.0006288016447797418,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from pickle import load
from sklearn.preprocessing import StandardScaler
import os

if(os.path.isfile(resp)):
    tRes = open(resp, 'wr')
else:
    tRes = open(resp, 'x')
scaler = StandardScaler()
x = scaler.fit_transform(dfRes.drop(columns=['file', 'frame']))
    
model = load(open(modelPath, 'rb'))
for i in range(len(x)):
    row = x[i]
    # print(dfRes.iloc[i][0])
    # print(dfRes.iloc[i][1])

    # print(dfRes.iloc[i][0])
    # print(model.predict([row])[0])

    # print(f'{dfRes.iloc[i][0]}\t{model.predict([row])[0]}\n')
    tRes.write(f'{dfRes.iloc[i][0]}\t{model.predict([row])[0]}\n')

tRes.close()

In [ ]:
import shutil
fList = 'D:\\SD\\yandexML\\pandModel\\data\\targets.tsv'
fNew = 'D:\\SD\\yandexML\\pandModel\\data\\testModel\\'
old = 'D:\\SD\\yandexML\\pandModel\\data\\trimmedAudio\\train\\'
i = 0

finSpectrList = []
finSexList = []
with open(fList, 'r') as targ:
    for ln in targ:
        audioName, sex = ln.split('\t')
        sex = sex.replace('\n','')
        pathToAudio = fNew+audioName+f'_{sex}.wav'
        oldName = old+audioName+'.wav'
        if(not os.path.exists(oldName)):
            i += 1
            continue
        i+=1
        print(oldName)
        print(pathToAudio)
        shutil.move(oldName, pathToAudio)
        if(i>200):
            break